In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/CS230/code")

In [ ]:
! pwd

/content/drive/MyDrive/CS230/code


In [3]:
! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 409 kB 4.0 MB/s 


In [4]:
!pip install segmentation_models_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.0 MB/s 
     |████████████████████████████████| 376 kB 24.3 MB/s 
     |████████████████████████████████| 58 kB 7.0 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=22b415cc10db83e227126f61a81596ffb11b7374b46457c2ac8dffd39d022330
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=4c23a112df6555e9a6904ea4479a153ce2c12b9ccc1866c2ffdd101e6984e3f1
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [5]:
logs = '/content/drive/MyDrive/CS230/checkpoint_resunet/runs'

In [6]:
from __future__ import print_function
from email.policy import default
from math import log10
from typing_extensions import Required
import numpy as np
import random
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from torch.autograd import Variable
from torch.utils.data import DataLoader
# from BSDDataLoader import get_training_set,get_test_set
# from Dataloader import RoadsDataset
from utils.data_load import *
from torchmetrics import JaccardIndex
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir = logs)
from models.res_unet import ResUnet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/CS230/checkpoint_resunet/runs

In [7]:
def map01(tensor,eps=1e-5):
    #input/output:tensor
    max = np.max(tensor.numpy(), axis=(1,2,3), keepdims=True)
    min = np.min(tensor.numpy(), axis=(1,2,3), keepdims=True)
    if (max-min).any():
        return torch.from_numpy( (tensor.numpy() - min) / (max-min + eps) )
    else:
        return torch.from_numpy( (tensor.numpy() - min) / (max-min) )

In [8]:
def sizeIsValid(size):
    for i in range(4):
        size -= 4
        if size%2:
            return 0
        else:
            size /= 2
    for i in range(4):
        size -= 4
        size *= 2
    return size-4

In [9]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super().__init__()

    def forward(self, input, target):
        pred = input.view(-1)
        truth = target.view(-1)

        # BCE loss
        # bce_loss = nn.BCELoss()(pred, truth).double()

        # Dice Loss
        dice_coef = (2.0 * (pred * truth).double().sum() + 1) / (
            pred.double().sum() + truth.double().sum() + 1
        )

        return (1 - dice_coef)

In [10]:
def main(args):
    target_size = sizeIsValid(args.size)
    print("outputsize is: "+str(target_size))
    if not target_size:
        raise  Exception("input size invalid")
    target_gap = (args.size - target_size)//2
    cuda = args.cuda
    if cuda and not torch.cuda.is_available():
        raise Exception("No GPU found, please run without --cuda")

    torch.manual_seed(args.seed)
    if cuda:
        torch.cuda.manual_seed(args.seed)

    print('===> Loading datasets')

    training_data_loader, valid_data_loader, testing_data_loader = load_datasets()
    print('===> Building ResUnet')
    unet = ResUnet(args.colordim)

    class_weights=torch.tensor([9],dtype=torch.float)
    if cuda:
        class_weights = class_weights.to('cuda')
        
    criterion = DiceLoss()
    if cuda:
        unet = unet.cuda()
        criterion = criterion.cuda()

    if args.Load_weight:
        unet.load_state_dict(torch.load(args.weight_path))
        print("===> Pretrained Weights loaded")
    optimizer = torch.optim.Adam(unet.parameters(), lr=args.lr)
    # decay LR
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

    def validation(epoch):
        totalloss = 0
        totaliou = 0
        for batch in valid_data_loader:
            input = Variable(batch[0],volatile=True)
            input = torch.permute(input, (0, 3, 1, 2))
            input = input.type(torch.float32)

            target = Variable(batch[1], volatile = True)
            target = torch.permute(target, (0, 3, 1, 2))
            target = target[:,1:2,:,:]
            target = target.type(torch.float32)
            if cuda:
                input = input.cuda()
                target = target.cuda()
            prediction = unet(input)
            
            loss = criterion(prediction, target)
            totalloss += loss.data

            jaccard = JaccardIndex(num_classes=2)
            if cuda:
                jaccard = jaccard.to('cuda')
            target = target.type(torch.int8)
            prediction[prediction >= 0.5] = 1
            prediction[prediction < 0.5] = 0
            prediction = prediction.type(torch.int8)
            if cuda:
                prediction = prediction.to('cuda')
                target = target.to('cuda')
            iou = jaccard(prediction, target)
            totaliou += iou.data
        print("===> Epoch[{}]: Avg. validation loss: {:.4f} dB".format(epoch, totalloss / len(testing_data_loader)))
        print("===> Avg. test iou: {:.4f} dB".format(totaliou / len(testing_data_loader)))


    def train(epoch):
        epoch_loss = 0
        totaliou = 0

        for iteration, batch in enumerate(training_data_loader, 1):
            input = Variable(batch[0],volatile=True)
            input = torch.permute(input, (0, 3, 1, 2))
            input = input.type(torch.float32)
            
            target = Variable(batch[1], volatile = True)
            target = torch.permute(target, (0, 3, 1, 2))
            target = target[:,1:2,:,:]
            target = target.type(torch.float32)

            if cuda:
                input = input.to('cuda')
                target = target.to('cuda')

            output = unet(input)
            loss = criterion(output, target)
            
            epoch_loss += loss.data
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            jaccard = JaccardIndex(num_classes=2)
            if cuda:
                jaccard = jaccard.to('cuda')
            target = target.type(torch.int8)
            output[output >= 0.5] = 1
            output[output < 0.5] = 0
            output = output.type(torch.int8)

            if cuda:
                output = output.to('cuda')
                target = target.to('cuda')
            iou = jaccard(output, target)
            totaliou += iou.data


            if iteration%10 == 0:
                writer.add_scalar("Loss/train", loss, (epoch-1)*270+iteration)
                writer.add_scalar("IOU/train", iou, (epoch-1)*270+iteration)
                print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch+args.current_trained_epochs, iteration, len(training_data_loader), loss.data))
            
        print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch+args.current_trained_epochs, epoch_loss / len(training_data_loader)))
        print("===> Epoch {} Complete: Avg. iou: {:.4f}".format(epoch+args.current_trained_epochs, totaliou / len(training_data_loader)))

    def test():
        totalloss = 0
        totaliou = 0
        for batch in testing_data_loader:
            input = Variable(batch[0],volatile=True)
            input = torch.permute(input, (0, 3, 1, 2))
            input = input.type(torch.float32)

            target = Variable(batch[1], volatile = True)
            target = torch.permute(target, (0, 3, 1, 2))
            target = target[:,1:2,:,:]
            target = target.type(torch.float32)
            if cuda:
                input = input.to('cuda')
                target = target.to('cuda')
            prediction = unet(input)
            
            loss = criterion(prediction, target)
            totalloss += loss.data

            jaccard = JaccardIndex(num_classes=2)
            if cuda:
                jaccard = jaccard.to('cuda')
            target = target.type(torch.int8)
            prediction[prediction >= 0.5] = 1
            prediction[prediction < 0.5] = 0
            prediction = prediction.type(torch.int8)
            if cuda:
                prediction = prediction.to('cuda')
                target = target.to('cuda')
            iou = jaccard(prediction, target)
            totaliou += iou.data
            
        print("===> Avg. test loss: {:.4f} dB".format(totalloss / len(testing_data_loader)))
        print("===> Avg. test iou: {:.4f} dB".format(totaliou / len(testing_data_loader)))


    def checkpoint(epoch):
        model_out_path = "/content/drive/MyDrive/CS230/checkpoint_resunet1/model_epoch_{}.pth".format(epoch+args.current_trained_epochs)
        torch.save(unet.state_dict(), model_out_path)
        print("Checkpoint saved to {}".format(model_out_path))
    
    for epoch in range(1, 1+args.nEpochs + 1):
        train(epoch)
        lr_scheduler.step()
        checkpoint(epoch)
        if epoch % 5 == 0:
          validation(epoch)
    test()
    writer.flush()
    writer.close()

In [ ]:
import argparse

p = argparse.ArgumentParser()
p.add_argument("--cuda", default=True)
p.add_argument("--batchSize", default=4,type=int)
p.add_argument("--testBatchSize", default=4,type=int)
p.add_argument("--lr", default=0.0000005, type=float)
# p.add_argument("--gamma", default=0.9, type=float)
# p.add_argument("--period", default=3,type=int)
p.add_argument("--nEpochs", default=50,type=int)
p.add_argument("--threads", default=5,type=int)
p.add_argument("--seed", default=123,type=int)
p.add_argument("--size", default=428,type=int)
p.add_argument("--remsize", default=20,type=int)
p.add_argument("--colordim", default=3,type=int)
p.add_argument("--Load_weight", default = False)
p.add_argument("--weight_path",default = '/content/drive/MyDrive/CS230/checkpoint_resunet1/model_epoch_55.pth')
p.add_argument("--current_trained_epochs", default = 0, type=int)
p.add_argument('-f')
# p.add_argument("--mode", Required=True)
args = p.parse_args()
main(args)

outputsize is: 244.0
===> Loading datasets
All dataset classes and their corresponding RGB values in labels:
Class Names:  ['background', 'road']
Class RGB values:  [[0, 0, 0], [255, 255, 255]]
Selected classes and their corresponding RGB values in labels:
Class Names:  ['background', 'road']
Class RGB values:  [[0, 0, 0], [255, 255, 255]]
1108
===> Building ResUnet
